<a href="https://colab.research.google.com/github/kbd95/Deep-Learning/blob/main/MNIST_TF_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# label > 5 =>> 1
# label <= 5 =>> 0

In [47]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [49]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [50]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
________________________________________________

In [53]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 3s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/10
1719/1719 - 3s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/10
1719/1719 - 3s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/10
1719/1719 - 3s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/10
1719/1719 - 3s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006
Epoch 6/10
1719/1719 - 3s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052
Epoch 7/10
1719/1719 - 3s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102
Epoch 8/10
1719/1719 - 3s - loss: 0.3485 - accuracy: 0.9022 - val_loss: 0.3209 - val_accuracy: 0.9138
Epoch 9/10
1719/1719 - 3s - loss: 0.3356 - accuracy: 0.9053 - val_loss: 0.3111 - val_accuracy: 0.9152
Epoch 10/10
1719/1719 - 3s - loss: 0.3251 - accuracy: 0.9077 - val_loss: 0.3016 - 

In [54]:
model.save("pretrained_mnist_model.h5")

# Transfer learning

In [55]:
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

In [56]:
pretrained_mnist_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
________________________________________________

In [57]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten_1: True
dense_7: True
leaky_re_lu_2: True
dense_8: True
leaky_re_lu_3: True
dense_9: True


In [58]:
a = [1,2,3,4,10]
a[0:-1]

[1, 2, 3, 4]

In [59]:
for layer in pretrained_mnist_model.layers[:-1]:
  layer.trainable = False
  print(f"{layer.name}: {layer.trainable}")

flatten_1: False
dense_7: False
leaky_re_lu_2: False
dense_8: False
leaky_re_lu_3: False


In [60]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten_1: False
dense_7: False
leaky_re_lu_2: False
dense_8: False
leaky_re_lu_3: False
dense_9: True


In [61]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(1, activation="sigmoid")
)

In [62]:
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 300)               235500    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 265,701
Trainable params: 101
Non-trainable params: 265,600
______________________________________________

In [63]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [64]:
new_model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [65]:
def update_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label > 5, 1, 0)
  return labels

In [66]:
y_train_bin, y_test_bin, y_valid_bin = update_labels([y_train, y_test, y_valid])

In [79]:
history = new_model.fit(X_train, y_train_bin, epochs=15,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/15
1719/1719 - 3s - loss: 0.3369 - accuracy: 0.8689 - val_loss: 0.3207 - val_accuracy: 0.8782
Epoch 2/15
1719/1719 - 3s - loss: 0.3337 - accuracy: 0.8704 - val_loss: 0.3173 - val_accuracy: 0.8788
Epoch 3/15
1719/1719 - 3s - loss: 0.3308 - accuracy: 0.8714 - val_loss: 0.3144 - val_accuracy: 0.8792
Epoch 4/15
1719/1719 - 3s - loss: 0.3283 - accuracy: 0.8722 - val_loss: 0.3125 - val_accuracy: 0.8804
Epoch 5/15
1719/1719 - 3s - loss: 0.3260 - accuracy: 0.8729 - val_loss: 0.3094 - val_accuracy: 0.8790
Epoch 6/15
1719/1719 - 3s - loss: 0.3239 - accuracy: 0.8737 - val_loss: 0.3075 - val_accuracy: 0.8814
Epoch 7/15
1719/1719 - 3s - loss: 0.3220 - accuracy: 0.8741 - val_loss: 0.3053 - val_accuracy: 0.8810
Epoch 8/15
1719/1719 - 3s - loss: 0.3201 - accuracy: 0.8751 - val_loss: 0.3036 - val_accuracy: 0.8804
Epoch 9/15
1719/1719 - 3s - loss: 0.3185 - accuracy: 0.8759 - val_loss: 0.3019 - val_accuracy: 0.8812
Epoch 10/15
1719/1719 - 3s - loss: 0.3170 - accuracy: 0.8762 - val_loss: 0.3001 - 

In [80]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.3034 - accuracy: 0.8834


[0.3034229874610901, 0.883400022983551]

In [81]:
X_new = X_test[:5]

y_test[:5], y_test_bin[:5]

(array([7, 2, 1, 0, 4], dtype=uint8), array([1, 0, 0, 0, 0]))

In [83]:
np.round(new_model.predict(X_new))

array([[1.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)